In [67]:
import torch
import time
from snn_delays.snn import SNN
from snn_delays.utils.dataset_loader import DatasetLoader
from snn_delays.utils.train_utils import get_device, print_spike_info, propagate_batch, set_seed
from snn_delays.utils.visualization_utils import plot_raster
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Subset, DataLoader
from collections import OrderedDict
device = get_device()
from torch.utils.data import Dataset, DataLoader
from tonic import MemoryCachedDataset
# for reproducibility
set_seed(42)

# Custom dataset for digit concatenation from a filtered dataset
class SequentialMemoryRetrievalDataset(Dataset):
    def __init__(self, base_dataset, sequence_length, target_classes):
        self.base_dataset = base_dataset
        indices = [i for i, (img, label) in enumerate(base_dataset) if np.argmax(label) in target_classes]
        zero_indices = [i for i, (img, label) in enumerate(base_dataset) if np.argmax(label) == 0]
        
        self.filtered_dataset = Subset(base_dataset, indices)
        self.zeros_dataset = Subset(base_dataset, zero_indices)

        self.indices = list(range(len(self.filtered_dataset)))  # Indices of the base dataset
        self.zeros_indices = list(range(len(self.zeros_dataset)))

        self.target_classes = target_classes
        #self.pairs = [(i, j) for i in self.indices for j in self.indices]  # All possible pairs of indices
        self.sequence_length = sequence_length
        #self.pairs = list(product(self.indices, repeat=sequence_length))
        self.num_classes = len(target_classes) # 0 is not a class 
        self.total_combinations = self.num_classes ** 2

    def __len__(self):
        # Number of pairs
        #return len(self.pairs)
        return len(self.base_dataset)

    def __getitem__(self, idx):
        # Get the indices for the current pair
        #indices = self.pairs[idx]
        # Retrieve the images and labels from the base dataset

        images = []
        all_labels = []
        retrieval_labels = []

        for i in range(self.sequence_length-2):
            img, label = self.filtered_dataset[np.random.choice(self.indices)]
            images.append(img)
            all_labels.append(self.target_classes.index(np.argmax(label)))

        # inserts zeroes randomly ()
        idx_frst_zero = np.random.randint(self.sequence_length//2)
        img, _ = self.zeros_dataset[np.random.choice(self.zeros_indices)]
        images.insert(idx_frst_zero, img)
        retrieval_labels.append(all_labels[idx_frst_zero])

        idx_scnd_zero = np.random.randint(idx_frst_zero+2, self.sequence_length-1)
        img, _ = self.zeros_dataset[np.random.choice(self.zeros_indices)]
        images.insert(idx_scnd_zero, img)
        retrieval_labels.append(all_labels[idx_scnd_zero-1])

        # Concatenate the images along the width (you can adjust as needed)
        concatenated_img = np.concatenate(images, axis=0)
        
        # Concatenate the labels one-hot
        encoded_label = sum(l * (self.num_classes ** i) for i, l in enumerate(reversed(retrieval_labels)))
        concatenated_label = np.zeros(self.total_combinations)
        concatenated_label[encoded_label] = 1.0

        return concatenated_img, concatenated_label


Running on: cuda:0


In [66]:
sequence_length = 10

for rpts in range(3):
    a = list(range(sequence_length-2))
    b = a.copy()
    idx_frst_zero = np.random.randint(sequence_length//2)
    a.insert(idx_frst_zero, 99)
    print((b[idx_frst_zero]))
    idx_scnd_zero = np.random.randint(idx_frst_zero+2, sequence_length-1)
    a.insert(idx_scnd_zero, 99)
    print((b[idx_scnd_zero-1]))
    idx_frst_zero, idx_scnd_zero
    print(a)
    
    

3
5
[0, 1, 2, 99, 3, 4, 99, 5, 6, 7]
3
6
[0, 1, 2, 99, 3, 4, 5, 99, 6, 7]
0
4
[99, 0, 1, 2, 3, 99, 4, 5, 6, 7]


In [68]:
device = get_device()

# for reproducibility
torch.manual_seed(10)

dataset = 'stmnist'
total_time = 5
batch_size = 128

# DATASET
DL = DatasetLoader(dataset=dataset,
                  caching='memory',
                  num_workers=0,
                  batch_size=batch_size,
                  total_time=total_time)

train_loader, test_loader, dataset_dict = DL.get_dataloaders()

target_classes = [1, 3, 8]
test_dataset = DL._dataset.test_dataset
train_dataset = DL._dataset.train_dataset

sequence_length = 10

conc_test_dataset = SequentialMemoryRetrievalDataset(test_dataset, sequence_length, target_classes)
conc_train_dataset = SequentialMemoryRetrievalDataset(train_dataset, sequence_length, target_classes)

Running on: cuda:0
[ToFrame(sensor_size=(10, 10, 2), time_window=None, event_count=None, n_time_bins=5, n_event_bins=None, overlap=0, include_incomplete=False)]


In [70]:
len(conc_test_dataset)

1390

In [71]:
train_dataset = MemoryCachedDataset(conc_train_dataset)
test_dataset = MemoryCachedDataset(conc_test_dataset)

train_loader = DataLoader(train_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=0)

test_loader = DataLoader(test_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=False,
                            pin_memory=True,
                            num_workers=0)

dataset_dict["num_output"] = conc_train_dataset.total_combinations
dataset_dict["num_training_samples"] = len(conc_train_dataset)
dataset_dict["time_ms"] = sequence_length*2e3 

print(dataset_dict)

{'num_input': 200, 'num_training_samples': 5562, 'num_output': 9, 'dataset_name': 'stmnist', 'time_ms': 20000.0, 'class_names': ['zero, one, two, three, four, five, six, seven, eight, nine']}


Training without delays (23%)

In [72]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 100

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
delay = None
#delay = (45, 15)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 4), connection_type='f',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
20000.0
Delta t: 400.0 ms
mean of normal: -19.999999997938847
training dvs5_seq10_Nonestmnist50_l4_1d1.t7 for 100 epochs...
Epoch [1/100], learning_rates 0.001000, 0.100000


c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
c:\Users\Alberto\anaconda3\envs\deepsnn\Lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to re

Step [14/43], Loss: 2.19105
l1_score: 0
Step [28/43], Loss: 2.20801
l1_score: 0
Step [42/43], Loss: 2.20095
l1_score: 0
Time elasped: 51.416489124298096
Test Loss: 2.201493891802701
Avg spk_count per neuron for all 50 time-steps 1.8639817237854004
Avg spk per neuron per layer [20.171065647482013, 7.139500899280575, 2.1508543165467624, 0.36228642086330937]
Test Accuracy of the model on the test samples: 11.151

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_3': 0.0000
Gradient norm for 'tau_m_4': 0.0000
Gradient norm for 'tau_m_o': 0.0048
Gradient norm for 'f0_f1.weight': 0.0005
Gradient norm for 'f1_f2.weight': 0.0011
Gradient norm for 'f2_f3.weight': 0.0078
Gradient norm for 'f3_f4.weight': 0.0703
Gradient norm for 'f4_o.weight': 0.1798
saving max acc: 11.151079136690647
Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD 

KeyboardInterrupt: 

Delays (50,5) 27%

In [73]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 100

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (50, 5)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45])

[INFO] Delays o: tensor([0])
20000.0
Delta t: 400.0 ms
mean of normal: -19.999999997938847
training dvs5_seq10_(50, 5)stmnist50_l2_50d5.t7 for 100 epochs...
Epoch [1/100], learning_rates 0.001000, 0.100000
Step [14/43], Loss: 2.50934
l1_score: 0
Step [28/43], Loss: 2.42668
l1_score: 0
Step [42/43], Loss: 2.31719
l1_score: 0
Time elasped: 11.45155954360962
Test Loss: 2.290514729239724
Avg spk_count per neuron for all 50 time-steps 4.675559043884277
Avg spk per neuron per layer [14.346335431654676, 4.355901528776978]
Test Accuracy of the model on the test samples: 11.511

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_o': 0.0318
Gradient norm for 'f0_f1.weight': 0.0381


Delays (50,1) 33%

In [74]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 100

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
#delay = None
delay = (50, 1)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='f',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

[INFO] Delays o: tensor([0])
20000.0
Delta t: 400.0 ms
mean of normal: -19.999999997938847
training dvs5_seq10_(50, 1)stmnist50_l2_50d1.t7 for 100 epochs...
Epoch [1/100], learning_rates 0.001000, 0.100000
Step [14/43], Loss: 3.26337
l1_score: 0
Step [28/43], Loss: 2.52973
l1_score: 0
Step [42/43], Loss: 2.24464
l1_score: 0
Time elasped: 10.300613164901733
Test Loss: 2.241960352117365
Avg spk_count per neuron for all 50 time-steps 4.605359077453613
Avg spk per neuron per layer [10

KeyboardInterrupt: 

Recurrent: 

In [ ]:
from snn_delays.utils.train_utils import train
from snn_delays.utils.test_behavior import tb_save_max_last_acc

num_epochs = 100

lr = 1e-3

taimu1 = time.time()

tau_m = 'normal'
#delay = (48*2,16*2)
#delay = (96,32)
delay = None
#delay = (50, 1)
ckpt_dir = 'exp21_STMNIST_memory' 

snn = SNN(dataset_dict=dataset_dict, structure=(64, 2), connection_type='r',
    delay=delay, delay_type='h', tau_m = tau_m,
    win=total_time*sequence_length, loss_fn='mem_sum', batch_size=batch_size, device=device,
    debug=False)

snn.set_network()

snn.model_name = f'dvs5_seq{sequence_length}_{delay}'+ snn.model_name

snn.to(device)
train(snn, train_loader, test_loader, lr, num_epochs, dropout=0.0, 
    test_behavior=tb_save_max_last_acc, ckpt_dir=ckpt_dir, scheduler=(100, 0.95), test_every=1)

print(f'[INFO] TIEMPO: {time.time() - taimu1}', flush=True)


[INFO] Delays: tensor([0])

[INFO] Delays i: tensor([0])

[INFO] Delays h: tensor([0])

[INFO] Delays o: tensor([0])
20000.0
Delta t: 400.0 ms
mean of normal: -19.999999997938847
training dvs5_seq10_Nonestmnist50_l2_1d1.t7 for 100 epochs...
Epoch [1/100], learning_rates 0.001000, 0.100000
Step [14/43], Loss: 2.62657
l1_score: 0
Step [28/43], Loss: 2.39892
l1_score: 0
Step [42/43], Loss: 2.25429
l1_score: 0
Time elasped: 16.1252863407135
Test Loss: 2.2370166778564453
Avg spk_count per neuron for all 50 time-steps 6.971970558166504
Avg spk per neuron per layer [18.74234487410072, 9.145537320143886]
Test Accuracy of the model on the test samples: 12.734

Model saved in  C:\Users\Alberto\OneDrive - UNIVERSIDAD DE SEVILLA\PythonData\Checkpoints\exp21_STMNIST_memory
Gradient norm for 'tau_m_1': 0.0000
Gradient norm for 'tau_m_2': 0.0000
Gradient norm for 'tau_m_o': 0.0268
Gradient norm for 'f0_f1.weight': 0.0615
Gradient norm for 'f1_f1.weight': 0.0058
Gradient norm for 'f1_f2.weight': 0.28